In [15]:
import pandas as pd
import numpy as np
import math  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, GlobalMaxPool1D, LayerNormalization, Dropout, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TextVectorization
import matplotlib.pyplot as plt

### 데이터 전처리

In [16]:
train_df = pd.read_csv('../data/train.csv', index_col=0)
gen_df = pd.read_csv('../data/gen_data_final.csv', index_col=0)
class_dict = {'협박 대화': 0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}

In [17]:
train_df

,class,conversation
idx,,
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...


In [18]:
gen_df

,conversation
topic,
오늘 하루 있었던 일,퇴근길이네요! 오늘 하루 수고 많으셨어요.\n네 수고하셨습니다. 특별한 일은 없으셨...
오늘 하루 있었던 일,점심 뭐 드셨어요?\n비빔밥 먹었어요. 맛있더라고요.\n오 저도 비빔밥 좋아하는데!...
오늘 하루 있었던 일,아침에 지하철이 너무 붐벼서 힘들었어요.\n정말요? 저는 버스 탔는데 그것도 만원이...
오늘 하루 있었던 일,날씨가 정말 좋네요. 점심시간에 공원 산책 다녀오셨어요?\n네 잠깐 회사 앞 올림픽...
오늘 하루 있었던 일,오늘 커피 맛이 평소보다 더 좋았던 것 같아요.\n정말요? 저는 오늘따라 커피가 쓰...
...,...
경험/추억,너 중학교 때 HOT 팬 아니었어? 방에 브로마이드 붙여놓고 그랬잖아.\n헐 어떻게...
경험/추억,대학교 1학년 때 갔던 MT 기억나? 술 마시고 게임하다가 필름 끊겼잖아.\n와 그...
경험/추억,나 어릴 때 살던 동네 골목길 사진을 우연히 봤는데 기분이 이상하더라.\n많이 변했...


In [19]:
gen_df['topic'] = '일반 대화'
gen_df = gen_df.rename(columns={'topic':'class'})

In [20]:
data_df = pd.concat([train_df, gen_df], ignore_index=True)

In [21]:
data_df

,class,conversation
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
4937,일반 대화,너 중학교 때 HOT 팬 아니었어? 방에 브로마이드 붙여놓고 그랬잖아.\n헐 어떻게...
4938,일반 대화,대학교 1학년 때 갔던 MT 기억나? 술 마시고 게임하다가 필름 끊겼잖아.\n와 그...
4939,일반 대화,나 어릴 때 살던 동네 골목길 사진을 우연히 봤는데 기분이 이상하더라.\n많이 변했...
4940,일반 대화,2002 월드컵 때 거리 응원 나갔던 거 생각하면 아직도 소름 돋아.\n와 맞아! ...


In [22]:
data_df['class'] = data_df['class'].apply(lambda x: class_dict[x])

In [23]:
data_df

,class,conversation
0,0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,0,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,3,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,1,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
4937,4,너 중학교 때 HOT 팬 아니었어? 방에 브로마이드 붙여놓고 그랬잖아.\n헐 어떻게...
4938,4,대학교 1학년 때 갔던 MT 기억나? 술 마시고 게임하다가 필름 끊겼잖아.\n와 그...
4939,4,나 어릴 때 살던 동네 골목길 사진을 우연히 봤는데 기분이 이상하더라.\n많이 변했...
4940,4,2002 월드컵 때 거리 응원 나갔던 거 생각하면 아직도 소름 돋아.\n와 맞아! ...


In [24]:
# train 데이터의 최대 길이를 구함
data_len = [len(x.split()) for x in data_df['conversation']]
MAX_LEN = max(data_len)
MAX_LEN

456

### 모델

In [57]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

In [58]:
MODEL_NAME = "klue/bert-base"

In [59]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [60]:
encoded_inputs = tokenizer(
    list(data_df['conversation']),
    padding='max_length', # 또는 padding=True (배치 내 최대 길이에 맞춤)
    truncation=True,
    max_length=MAX_LEN,       # BERT 모델이 처리 가능한 최대 길이 고려 (klue/bert-base는 512)
    return_tensors='tf'
)

In [66]:
labels = tf.constant(data_df['class'].values)
unique_labels = np.unique(labels.numpy())
NUM_CLASSES = len(unique_labels) # 전체 클래스 갯수

In [67]:
labels, unique_labels, NUM_CLASSES

(<tf.Tensor: shape=(4942,), dtype=int64, numpy=array([0, 0, 3, ..., 4, 4, 4])>,
 array([0, 1, 2, 3, 4]),
 5)

In [68]:
num_samples = len(data_df) # 전체 샘플 갯수
indices = np.arange(num_samples) # 인덱스 생성

train_indices, val_indices = train_test_split( # 인덱스를 8대2로 나눔
    indices,
    test_size=0.2,
    random_state=42,
    stratify=labels.numpy() # stratify에는 target값으로 class 비율 일정하게 셔플
)

In [69]:
train_inputs = {key: tf.gather(val, train_indices) for key, val in encoded_inputs.items()}
val_inputs = {key: tf.gather(val, val_indices) for key, val in encoded_inputs.items()}

# 레이블도 동일한 인덱스로 선택
train_labels = tf.gather(labels, train_indices)
val_labels = tf.gather(labels, val_indices)

In [70]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels))
train_dataset = train_dataset.shuffle(len(train_indices)).batch(16) # 셔플 및 배치

# 예시: 검증 데이터셋 생성
val_dataset = tf.data.Dataset.from_tensor_slices((val_inputs, val_labels))
val_dataset = val_dataset.batch(16) # 검증 데이터는 보통 셔플하지 않음

In [74]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
    # 각도 배열 생성
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)

        # 배열의 짝수 인덱스에는 sin 함수 적용
        sines = tf.math.sin(angle_rads[:, 0::2])
        # 배열의 홀수 인덱스에는 cosine 함수 적용
        cosines = tf.math.cos(angle_rads[:, 1::2])

        # sin과 cosine이 교차되도록 재배열
        pos_encoding = tf.stack([sines, cosines], axis=0)
        pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
        pos_encoding = tf.reshape(pos_encoding, [position, d_model])

        pos_encoding = pos_encoding[tf.newaxis, ...]

        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [75]:
# 스케일드 닷 프로덕트 어텐션 함수

def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
    matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
    if mask is not None:
        logits += (mask * -1e9)

  # softmax적용
    attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
    output = tf.matmul(attention_weights, value)
    return output

In [76]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # Q, K, V에 각각 Dense를 적용합니다
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # 병렬 연산을 위한 머리를 여러 개 만듭니다
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # 스케일드 닷 프로덕트 어텐션 함수 사용
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # 최종 결과에도 Dense를 한 번 더 적용합니다
        outputs = self.dense(concat_attention)

        return outputs

In [77]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [78]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [79]:
# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    # 두 번째 서브 레이어 : 2개의 완전연결층
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [80]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    # 패딩 마스크 사용
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    # 임베딩 레이어
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    # 포지셔널 인코딩
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    # num_layers만큼 쌓아올린 인코더의 층.
    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [89]:
def decoder_layer_no_cross(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")

    # 1. 셀프 어텐션만 수행
    attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")(
        inputs={'query': inputs, 'key': inputs, 'value': inputs, 'mask': look_ahead_mask}
    )
    attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)

    # 2. 피드포워드
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention1)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention1)

    return tf.keras.Model(inputs=[inputs, look_ahead_mask], outputs=outputs, name=name)

In [90]:
def transformer_with_decoder_classifier(vocab_size,
                                        num_layers,
                                        units,
                                        d_model,
                                        num_heads,
                                        dropout,
                                        num_classes,
                                        name='transformer_with_decoder_classifier'):
    # 입력
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    # === 인코더 ===
    embeddings_enc = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings_enc *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings_enc = PositionalEncoding(vocab_size, d_model)(embeddings_enc)
    enc_outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings_enc)

    for i in range(num_layers):
        enc_outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name=f"encoder_layer_{i}"
        )([enc_outputs, padding_mask])

    # === 디코더 === (교차 어텐션 제거된 버전)
    embeddings_dec = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings_dec *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings_dec = PositionalEncoding(vocab_size, d_model)(embeddings_dec)
    dec_outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings_dec)

    for i in range(num_layers):
        dec_outputs = decoder_layer_no_cross(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name=f"decoder_layer_{i}"
        )([dec_outputs, look_ahead_mask])

    # === 분류 헤드 ===
    pooled_output = tf.keras.layers.GlobalAveragePooling1D()(dec_outputs)
    x = tf.keras.layers.Dense(units=units, activation='relu')(pooled_output)
    x = tf.keras.layers.Dropout(rate=dropout)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    return tf.keras.Model(
        inputs=[inputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name
    )

In [92]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer_with_decoder_classifier(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    num_classes=NUM_CLASSES,
    dropout=DROPOUT)

model.summary()

Model: "transformer_with_decoder_classifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    5120000     inputs[0][0]                     
__________________________________________________________________________________________________
tf.math.multiply_1 (TFOpLambda) (None, None, 256)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
positional_encoding_1 (Position (None, None, 256)    0           tf.math.multiply_1[0][0]         
________________________________________________________________

In [85]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [87]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [93]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [95]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping_cb = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
    patience=2)

model_checkpoint_cb = ModelCheckpoint(
    filepath='bert_pretrained.h5', # 저장 경로
    monitor='val_loss',           # 관찰 대상: 검증 손실
    save_best_only=True,          # 최고의 모델만 저장
    save_weights_only=True,       # 가중치만 저장
    mode='min',                   # val_loss는 낮을수록 좋으므로 'min'
    verbose=1                     # 저장 시 로그 출력
)

In [96]:
NUM_EPOCHS = 50
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=NUM_EPOCHS, # 충분한 에폭 수 지정 (조기 종료가 관리)
    callbacks=[early_stopping_cb, model_checkpoint_cb] # 정의된 콜백 전달
)

Epoch 1/50


ValueError: in user code:

    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.9/site-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /opt/conda/lib/python3.9/site-pack